In [ ]:
import moth
import os
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import csv
import scipy
import scipy.signal
import h5py

PATH = '/storage/home/adz6/group/project'
VNADATAPATH = os.path.join(PATH, 'datasets/data/lab/vna')
DAQDATAPATH = os.path.join(PATH, 'datasets/data/lab/dig')

LABDATAPATH = os.path.join(PATH, 'labdata')
#MOTHPATH = os.path.join(PATH, 'moth')
#MOTHDATA = os.path.join(MOTHPATH, 'data/datasets')
#PLOTPATH = os.path.join(PATH, 'plots/moth')
#MOTHRESULTS = os.path.join(PATH, 'results/moth')


def CombineMeasurements(name, path):
    
    
    h5file = h5py.File(name, 'w')
    
    params = ExtractSweepParameters(path)
    print(params)
    
    for i, meas_file in enumerate(os.listdir(path)):
        print(meas_file)
        h5file.create_group(f'{meas_file}',)
        h5file[f'{meas_file}'].create_dataset('array',  (params['nrad'][i], params['nangle'][i], params['nsample'][i]), dtype='c8')
        h5file[f'{meas_file}'].create_dataset('freq', (params['nsample'][i]), dtype='f8')
        h5file[f'{meas_file}'].__setitem__('step', params['step'][i])
        h5file[f'{meas_file}'].__setitem__('z', params['z'][i])
        
        angles = np.int32(np.arange(0, 360, 3))
        #print(params['nangle'][i])
        print(angles)
        
        for j, rad_sweep_file in enumerate(os.listdir(os.path.join(path, meas_file))):
            irad = int(rad_sweep_file)
            for k, ang_sweep_file in enumerate(os.listdir(os.path.join(path, meas_file, rad_sweep_file))):
                angle = int(ang_sweep_file.split('.')[0])
                #print(angle, rad_sweep_file)
                
                iangle = np.argwhere(angles == angle)[0, 0]
                
                data, freqs = ParseVNATrace(os.path.join(path, meas_file, rad_sweep_file, ang_sweep_file))
                h5file[f'{meas_file}']['array'][irad, iangle, :] = data
                h5file[f'{meas_file}']['freq'][:] = freqs
    h5file.close()
    
def ExtractSweepParameters(path):
    
    rad_step_list = []
    z_pos_list = []
    
    params = {'step': [], 'z': [], 'nrad': [], 'nangle': [], 'nsample': []}
    for i, file in enumerate(os.listdir(path)):
        params['step'].append(int(file.split('step')[-1].split('mm')[0])) #mm
        params['z'].append(int(file.split('z')[-1].split('mm')[0]))
        
        params['nrad'].append(len(os.listdir(os.path.join(path, file))))
        
        #print(os.listdir(os.path.join(path, file, '0')))
        
        params['nangle'].append(len(os.listdir(os.path.join(path, file, '0'))))
        
        example_file = os.listdir(os.path.join(path, file, '0'))[0]
        example_trace, example_freqs = ParseVNATrace(os.path.join(path, file, '0', example_file))
        
        params['nsample'].append(example_trace.size)
    
    return params

def ParseVNATrace(path):
    
    with open(path, 'r') as infile:
        for i, item in enumerate(csv.reader(infile)):
            if i == 0:
                item = np.float32(item)
                
                real = np.float32(item[np.arange(0, item.size, 2)])
                imag = np.float32(item[np.arange(1, item.size, 2)])
                
                trace = real + 1j * imag
            elif i == 1:
                start_f = np.float32(item[-1])
            elif i == 2:
                end_f = np.float32(item[-1])
                
    return trace, np.linspace(start_f, end_f, trace.size)
    
    
    

In [ ]:
os.listdir(os.path.join(LABDATAPATH, 'vna'))

In [ ]:
#print(os.listdir(os.path.join(LABDATAPATH, 'vna', '211119_cres2_vna_trace', 'range3cm_step1mm_z0mm')))

path2data = os.path.join(LABDATAPATH, 'vna', '211119_cres2_vna_trace')

name = '211220_cres2_vna_trace.h5'

CombineMeasurements(os.path.join(VNADATAPATH, name), path2data)

In [ ]:
os.listdir(VNADATAPATH)

In [ ]:
angles = np.arange(0, 360, 360 // 60)

In [ ]:
np.argwhere(angles == 12)[0, 0]

In [ ]:
np.linspace(0, 360, 121)[0:120]